In [1]:
# Import Pyself. Resources
from pyspark.sql.functions import *

# My Imports
import os
import shutil
from datetime import datetime
import hashlib

# PySpark Imports
from pyspark.sql import SparkSession

class MyFunctions(object):

    def __init__(self):
        self.spark = (SparkSession.builder
                      .config('spark.jars', 'driver/postgresql-42.6.0.jar')
                      .config('spark.driver.extraClassPath', 'driver/postgresql-42.6.0.jar')
                      .appName("MyProject").getOrCreate())

        self.host = "localhost"
        self.port = "5432"
        self.database = "ensurwave"
        self.username = "postgres"
        self.password = "postgres"
        self.url = f"jdbc:postgresql://{self.host}:{self.port}/{self.database}"

        self.file = 'data/new/20230331_employees_details.json'

In [2]:
mf = MyFunctions()

23/04/15 23:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/15 23:46:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Getting the schema
schema_raw_data = str(mf.spark.read.option('inferSchema',True).option('multiline','true').json(mf.file).schema)

current_no_space = schema_raw_data.replace(' ','')
current_no_space = [i for i in current_no_space]
current_no_space.sort()
current_no_space = ''.join(current_no_space)

current_hash_no_space = hashlib.sha256(current_no_space.encode('utf-8'))
current_hash_no_space = current_hash_no_space.hexdigest()

# Mounting the dataframe to save the history of schemas used.
data = [{'id' : 1, 'schema' : schema_raw_data, 'version' : current_hash_no_space, 'fileName':mf.file.split('/')[-1]}]
current_schema = mf.spark.createDataFrame(data)
current_schema = current_schema.select('id','schema','version','fileName').withColumn('load_timestamp', to_timestamp(lit(datetime.now().strftime("%Y%m%d %H:%M:%S")),'yyyyMMdd H:m:s'))
current_schema.show()

+---+--------------------+--------------------+--------------------+-------------------+
| id|              schema|             version|            fileName|     load_timestamp|
+---+--------------------+--------------------+--------------------+-------------------+
|  1|StructType([Struc...|f7a43c8956169f623...|20230331_employee...|2023-04-15 23:46:18|
+---+--------------------+--------------------+--------------------+-------------------+



In [4]:
# Get Current table in 
try:
    table_history = (mf.spark.read.format('jdbc').option('url', mf.url)
                        .option('dbtable', 'schema_history')
                        .option('user', mf.username)
                        .option('password', mf.password)
                        .option('driver', 'org.postgresql.Driver').load())

    max_id = table_history.select(max(col('id')).alias('max_id')).collect()[0][0]
    new_id = max_id + 1

    new_value = [{'id':new_id, 'schema':schema_raw_data, 'version' : current_hash_no_space, 'fileName':mf.file.split('/')[-1]}]

    df_new_value = mf.spark.createDataFrame(new_value)
    df_new_value = df_new_value.select('id','schema','version','fileName').withColumn('load_timestamp', to_timestamp(lit(datetime.now().strftime("%Y%m%d %H:%M:%S")),'yyyyMMdd H:m:s'))

    old_hash_schema = table_history.filter(f"id == ({max_id})").select('version').collect()[0][0]
    
    if old_hash_schema != current_hash_no_space:
        df_persist = table_history.unionAll(df_new_value)

        df_persist.write.mode('overwrite').saveAsTable('df_persist')

        tb_persist = mf.spark.table('df_persist').orderBy(col('id'))

        (tb_persist
            .write
            .mode('overwrite')
            .format('jdbc')
            .option('url', mf.url)
            .option('dbtable', 'schema_history')
            .option('user', mf.username)
            .option('password', mf.password)
            .option('driver', 'org.postgresql.Driver')
            .save())
        shutil.rmtree(f'{os.getcwd()}/spark-warehouse')
# # Creating Table
except:
    print('Nova Tabela')
    (current_schema
        .write
        .format('jdbc')
        .option('url', mf.url)
        .mode('overwrite')
        .option('dbtable', 'schema_history')
        .option('user', mf.username)
        .option('password', mf.password)
        .option('driver', 'org.postgresql.Driver')
        .save())

Nova Tabela


In [5]:
current = (mf.spark.read
    .format('jdbc')
    .option('url', mf.url)
    .option('dbtable', 'schema_history')
    .option('user', mf.username)
    .option('password', mf.password)
    .option('driver', 'org.postgresql.Driver')
    .load())
current.show()

+---+--------------------+--------------------+--------------------+-------------------+
| id|              schema|             version|            fileName|     load_timestamp|
+---+--------------------+--------------------+--------------------+-------------------+
|  1|StructType([Struc...|f7a43c8956169f623...|20230331_employee...|2023-04-15 23:46:18|
+---+--------------------+--------------------+--------------------+-------------------+



In [6]:
current.count()

1